In [32]:
import pandas

from matplotlib import pyplot
from deps.data import load_metadata
from deps.logger import logger
from deps.data import load_raw_data, load_data
from hcve_lib.functional import pipe
from hcve_lib.evaluation_functions import c_index
from hcve_lib.data import format_identifier_long

logger.setLevel('DEBUG')
pandas.set_option("display.max_columns", None)
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'
pyplot.rcParams['figure.facecolor'] = 'white'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
metadata = load_metadata()
data = load_data(metadata)
raw_data = load_raw_data()
raw_data_subset = raw_data.loc[data.index]

Raw data
	n cohorts=21
	n individuals=52,585

Baseline visit kept
 	n individuals removed=5,529

 	n individuals=47,056

HF cohorts removed
	n cohorts removed=3: hfgr, timechf, leitzaran
	n individuals removed=1,073

	n cohorts=18
	n individuals=45,983

No outcome cohorts removed
	n cohorts removed=5: biomarcoeurs, dyda, epath, iblomaved, stophf
	n individuals removed=3,077

	n cohorts=13
	n individuals=42,906

Missing HF data cohorts removed
	n cohorts removed=6: adelhyde, gecoh, r2c2, reve(1-2), stanislas, styrianvitd
	n individuals removed=6,209

	n cohorts=7
	n individuals=36,697

Missing blood pressure measurements
 	n individuals removed=434

 	n individuals=36,263

Providing missing PP for 3075individuals

HF individuals at baseline removed
 	n individuals removed=2,953

 	n individuals=33,310

Missing outcome individuals removed
 	n individuals removed=905

 	n individuals=32,405

Final dataset
	n individuals=32,405
	n cohorts=7



In [34]:
data = load_data(metadata)
data[data['FUNFHF'].isna()]

Raw data
	n cohorts=21
	n individuals=52,585

Baseline visit kept
 	n individuals removed=5,529

 	n individuals=47,056

HF cohorts removed
	n cohorts removed=3: hfgr, timechf, leitzaran
	n individuals removed=1,073

	n cohorts=18
	n individuals=45,983

No outcome cohorts removed
	n cohorts removed=5: biomarcoeurs, dyda, epath, iblomaved, stophf
	n individuals removed=3,077

	n cohorts=13
	n individuals=42,906

Missing HF data cohorts removed
	n cohorts removed=6: adelhyde, gecoh, r2c2, reve(1-2), stanislas, styrianvitd
	n individuals removed=6,209

	n cohorts=7
	n individuals=36,697

Missing blood pressure measurements
 	n individuals removed=434

 	n individuals=36,263

Providing missing PP for 3075individuals

HF individuals at baseline removed
 	n individuals removed=2,953

 	n individuals=33,310

Missing outcome individuals removed
 	n individuals removed=905

 	n individuals=32,405

Final dataset
	n individuals=32,405
	n cohorts=7



,IDNR,STUDY_NUM,STUDY,VISIT,SEX,AGE,BW,BH,BMI,SMK,DRK,DIABETES,HCV,HCAD,HPAD,HAF,HCEREBROV,TRT_AH,TRT_ACE,TRT_CCB,TRT_DIUR,TRT_BB,TRT_ARB,TRT_LIP,SBP,DBP,HR,PP,SOK,QRS,CI,GLU,CHOL,LDL,HDL,TG,CREA,FCV,FUFCV,NFHF,FUNFHF,FNFHF,FUFNFHF
IDNR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [35]:
for column in data.columns:
    print(f'{len(data[column].unique())}: {format_identifier_long(column, metadata)}')

32405: [IDNR] IDNR
7: [STUDY_NUM] Numeric study identifier
7: [STUDY] STUDY
1: [VISIT] VISIT
2: [SEX] Gender
16090: [AGE] Age
1878: [BW] Body weight
883: [BH] Body height
13216: [BMI] Body-mass index_frame
4: [SMK] Smoking status
3: [DRK] Alcohol intake
3: [DIABETES] History of diabetes
3: [HCV] History of cardiovascular disease
3: [HCAD] History of coronary artery disease
3: [HPAD] History of peripheral artery disease
3: [HAF] History of atrial fibrillation
3: [HCEREBROV] History of cerebrovascular incident
3: [TRT_AH] Intake of antihypertensive medication
3: [TRT_ACE] Intake of angiotensin converting enzyme inhibitors
3: [TRT_CCB] Intake of calcium channel blockers
3: [TRT_DIUR] Intake of diuretics
3: [TRT_BB] Intake of beta-blokcers
3: [TRT_ARB] Intake of angiotensin II receptor blockers
3: [TRT_LIP] Intake of lipid-lowering medication
471: [SBP] Systolic blood pressure
308: [DBP] Diastolic blood pressure
189: [HR] Heart rate
408: [PP] Pulse pressure
557: [SOK] Sokolow-Lyon index_fr

In [36]:

for column in data.columns:
    if len(data[column].unique()) < 10:
        data.loc[:, column] = data[column].astype('category')
    print(f'{data[column].dtype}: {format_identifier_long(column, metadata)}')

int64: [IDNR] IDNR
category: [STUDY_NUM] Numeric study identifier
category: [STUDY] STUDY
category: [VISIT] VISIT
category: [SEX] Gender
float64: [AGE] Age
float64: [BW] Body weight
float64: [BH] Body height
float64: [BMI] Body-mass index_frame
category: [SMK] Smoking status
category: [DRK] Alcohol intake
category: [DIABETES] History of diabetes
category: [HCV] History of cardiovascular disease
category: [HCAD] History of coronary artery disease
category: [HPAD] History of peripheral artery disease
category: [HAF] History of atrial fibrillation
category: [HCEREBROV] History of cerebrovascular incident
category: [TRT_AH] Intake of antihypertensive medication
category: [TRT_ACE] Intake of angiotensin converting enzyme inhibitors
category: [TRT_CCB] Intake of calcium channel blockers
category: [TRT_DIUR] Intake of diuretics
category: [TRT_BB] Intake of beta-blokcers
category: [TRT_ARB] Intake of angiotensin II receptor blockers
category: [TRT_LIP] Intake of lipid-lowering medication
float

In [37]:
import pickle

with open('./data/prediction.data', 'rb') as file:
    results = pickle.load(file)


/home/sitnarf/.local/share/virtualenvs/homage-fl-Ms4N7PDp/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator FunctionTransformer from version 0.24.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [ ]:
from hcve_lib.tracking import load_pickled_artifact
from hcve_lib.visualisation import b, h2
import mlflow
import yaml
from hcve_lib.data import format_identifier
from toolz.curried import valmap, partial
from toolz import valfilter, identity
from pandas import DataFrame
import plotly.express as px
from hcve_lib.serialization import to_json_serializable
from hcve_lib.evaluation_functions import compute_metric_groups, get_2_level_groups

for evaluation_type in ('reproduce', 'lco', 'lm'):
    h2(evaluation_type)
    runs = mlflow.search_runs('0')
    last_run = runs[runs['tags.mlflow.parentRunId'].isna() & (runs['params.cv'] == evaluation_type)].iloc[0]
    result = load_pickled_artifact(last_run['run_id'], 'result')

    removed_features = pipe(
        result['column_masks'],
        valmap(lambda mask: valfilter(identity, mask)),
        valmap(lambda mask: list(mask)),
        valmap(lambda mask: map(partial(format_identifier, metadata=metadata), mask)),
        valmap(lambda mask: list(mask)),
    )
    b('Removed features')
    print(yaml.dump(removed_features))
    metrics_group = compute_metric_groups(c_index, get_2_level_groups(result['predictions'], raw_data_subset.groupby('STUDY')))
    # for fold_name, fold in result['predictions'].items():
        # print(len(fold['model'][-1].coef_[0]))
        # print(Series(fold['model'][-1].coef_, index=fold['model'][-1].get_feature_names()))
    fig = px.imshow(
        DataFrame(metrics_group),
        labels=dict(x="Trained on", y="Tested on", color="c-index"),
    )
    fig.show()



In [ ]:
print(yaml.dump(to_json_serializable(metrics_group)))

# Hyperparams feature importance

In [50]:
import optuna
from mlflow import set_tracking_uri
from run_lco_optimized import METHODS_DEFINITIONS

set_tracking_uri('http://localhost:5000')

runs = get_completed_runs('lco_optimized')

for method_name in METHODS_DEFINITIONS.keys():
    h2(method_name)
    last_run = get_last_run(runs, is_root_run(runs) & (runs['tags.method_name'] == method_name))
    study = get_study(last_run['run_id'])
    print(study.best_trial.user_attrs['hyperparameters'])
    optuna.visualization.plot_optimization_history(study).show()
    # optuna.visualization.plot_edf(study).show()
    # optuna.visualization.plot_param_importances(result).show()
    # optuna.visualization.plot_parallel_coordinate(study).show()

{'estimator': {'l1_ratio': 0.001524599887137823}}


{'estimator': {'learning_rate': 0.7312075046699227, 'max_depth': 2, 'n_estimators': 95, 'min_samples_split': 27, 'min_samples_leaf': 137, 'max_features': 'log2', 'subsample': 0.40294583605008727}}


{'estimator': {'n_estimators': 23, 'max_depth': 19, 'min_samples_split': 12, 'min_samples_leaf': 158, 'max_features': 'log2', 'oob_score': True}}


IndexError: single positional indexer is out-of-bounds

In [56]:
runs = get_completed_runs('lco')
last_run = get_last_run(runs, is_root_run(runs) & (runs['tags.method_name'] == 'coxnet'))
result = load_pickled_artifact(last_run['run_id'], 'result')


OSError: No such file or directory: './mlruns/4/48a986c9cb9142fcb7f036c406ec0008/artifacts/result'

## Brier scores

In [ ]:
from sksurv.metrics import brier_score

for fold_name, fold in result['predictions'].items():
    b(fold_name)
    try:
        print(brier_score(
            fold['y_train'],
            fold['y_true'],
            [fn(365*5) for fn in fold['model'] \
                .predict_survival_function(fold['X_test'])],
            365*5,
        )[1])
    except Exception as e:
        print(e)

## Survival functions

In [ ]:
from numpy import arange

for fold_name, fold in result['predictions'].items():
    pyplot.figure()
    pyplot.title(fold_name)
    y_survival = fold['model'].predict_survival_function(fold['X_test'])
    for step_function in y_survival:
        step_x = list(arange(0, 365*5, step=1))
        pyplot.step(step_x, step_function(step_x), where="post", c='navy', alpha=0.05)
        pyplot.gca().ticklabel_format(useOffset=False, style='plain')
        pyplot.ylabel("est. probability of survival $\hat{S}(t)$")
        pyplot.xlabel("time $t$")
    pyplot.show()

In [ ]:
from deps.common import get_variables_cached
data, metadata, X, y = get_variables_cached()

In [161]:
from hcve_lib.visualisation import p
from optuna import Trial
from hcve_lib.tracking import get_completed_runs, get_run_duration
from hcve_lib.tracking import get_last_run
from hcve_lib.tracking import is_root_run
from hcve_lib.tracking import get_children_run
import humanize
import pytz
from datetime import datetime

set_tracking_uri('http://localhost:5000')
runs = get_completed_runs('lco_optimized')
last_run = get_last_run(runs, is_root_run(runs) & (runs['tags.mlflow.runName'] == 'coxnet'))
children = get_children_run(runs, last_run['run_id'])

def get_trial_duration_min(trial: Trial) -> float:
    return (trial.datetime_complete-trial.datetime_start).total_seconds()  / 60

for _, run in children.iterrows():
    b(run['tags.mlflow.runName'])
    p(humanize.naturaltime(datetime.now(tz=pytz.utc) - last_run['start_time']))
    p(f'Total duration: {humanize.naturaldelta(get_run_duration(run))}')
    study = load_pickled_artifact(run['run_id'], 'study')
    # optuna.visualization.plot_optimization_history(study).show()
    # optuna.visualization.plot_param_importances(result).show()
    # optuna.visualization.plot_parallel_coordinate(study).show()
    trial_durations = [
        get_trial_duration_min(trial) for trial in study.trials]
    px.scatter(trial_durations, labels={'index': 'Run', 'value': 'Duration'}).show()
    x = [trial.params['__cross_validate__missing_fraction'] for trial in study.trials]
    y = [trial.value for trial in study.trials]
    # plotly.express.scatter(x=x, y=y).show()

In [163]:
runs.keys()

Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.c_index_l', 'metrics.c_index', 'metrics.c_index_r',
       'metrics.brier_r', 'metrics.brier', 'metrics.brier_l', 'metrics.value',
       'params.metrics', 'params.hyperparameters', 'params.cv_hyperparameters',
       'params.estimator_n_alphas', 'params.estimator_max_depth',
       'params.estimator_min_samples_split',
       'params.estimator_min_samples_leaf', 'params.estimator_max_features',
       'params.estimator_oob_score', 'params.estimator_n_estimators',
       'params.estimator_subsample', 'params.estimator_learning_rate',
       'tags.mlflow.source.name', 'tags.mlflow.source.git.commit',
       'tags.mlflow.runName', 'tags.mlflow.parentRunId', 'tags.mlflow.user',
       'tags.mlflow.source.type', 'tags.method_name', 'tags.datetime_complete',
       'tags.number', 'tags.state', 'tags.direction', 'tags.datetime_start',
       'tags.estimator_n_alphas_distribution', 'tags.hype